In [2]:
import yfinance as yf
import pandas as pd
import math
from tqdm import tqdm as tqdm

DOWNLOAD = True

In [3]:
tickers = ['TSLA',
           'AAPL',
           'MSFT',
           'U',
           'CCL',
           'TD',
           'SPY',
           'FB',
           'V',
           'DIS',
           'CNR',
           'WEED.TO',
           'SNC.TO',
           'SHOP',
           'SU.TO',
           'CM.TO',
           'TD.TO',
           'ENB.TO',
           'APHA.TO',
           'XIU.TO', # s&p/tsx composite 60
           'AC.TO']

In [4]:
# u = yf.Ticker("U")
# udata = u.history(period='2y', interval = '1h')

In [5]:
#tdata.loc[tdata['name']=='2021-01-22']
# tdata.tail(10)

In [6]:
# datee=udata.iloc[1]
# datee

In [7]:
# import pandas as pd
# datee.name == pd.Timestamp('2020-09-18')
# true

In [4]:
raws = {}

for ticker in tqdm(tickers):
    t = yf.Ticker(ticker)
    t_data = t.history(period = '10y', interval = '1d')
    
    raws[ticker] = t_data
    

100%|██████████| 21/21 [00:05<00:00,  3.83it/s]


In [5]:
col_d = {} # dropped columns

for ticker in tqdm(tickers):
    raw = raws[ticker]
    
    col_d[ticker] = raw.drop(columns=['Dividends', 'Stock Splits'])
    
col_d['TSLA'].head()

100%|██████████| 21/21 [00:00<00:00, 778.36it/s]


,Open,High,Low,Close,Volume
Date,,,,,
2011-01-27,4.948,5.016,4.906,4.984,4478500
2011-01-28,4.976,4.976,4.750,4.802,5242000
2011-01-31,4.810,4.824,4.700,4.820,4151500
2011-02-01,4.862,4.946,4.708,4.782,3539000
2011-02-02,4.832,4.836,4.734,4.788,2847500


In [6]:
import numpy as np

In [12]:
good = {}

for ticker in tqdm(tickers):
    df = col_d[ticker]
    df.dropna(inplace=True)  # drops NaN rows
    
    pct = df.pct_change()
        
    pct['DayOfWeek'] = [i.dayofweek/4 for i in df.index] # day of the week on [0,1]
    
    pct.dropna(inplace=True)
    
    good[ticker] = pct
    

print(good['TSLA'])

    

100%|██████████| 21/21 [00:00<00:00, 83.35it/s]

                Open      High       Low     Close    Volume  DayOfWeek
Date                                                                   
2011-01-27  0.001214  0.008039  0.017842  0.006869 -0.170571       0.75
2011-01-28  0.005659 -0.007974 -0.031798 -0.036517  0.170481       1.00
2011-01-31 -0.033360 -0.030547 -0.010526  0.003748 -0.208031       0.00
2011-02-01  0.010811  0.025290  0.001702 -0.007884 -0.147537       0.25
2011-02-02 -0.006170 -0.022240  0.005523  0.001255 -0.195394       0.50
...              ...       ...       ...       ...       ...        ...
2021-01-20  0.024994  0.011176  0.005138  0.006986  0.011783       0.50
2021-01-21 -0.004355 -0.004398  0.004945 -0.006420 -0.197453       0.75
2021-01-22 -0.024199 -0.009022 -0.015212  0.001953 -0.025808       1.00
2021-01-25  0.024799  0.061792  0.012310  0.040348  1.045065       0.00
2021-01-26  0.042550 -0.004998  0.039079  0.008833 -0.611280       0.25

[2516 rows x 6 columns]


In [13]:
import pickle

#serializing for future use

with open('./data/std-stocks.pkl', 'wb') as f:
    pickle.dump(good, f, protocol=pickle.HIGHEST_PROTOCOL)

## Variables
- Open (n)
- High (n)
- Low (n)
- Close (n)
- Volume (n)
- Day of Week (ex. weekends)


In [ ]:
# creating the 'answers'/future price
outData = pd.DataFrame(data={'Future':[]})

outData['Future'] = good['TSLA']['Close'].shift(-1)

print(good['TSLA']['Close'])

print(outData)

In [14]:
with_future = {}    
    
# adding future price
for ticker in tqdm(good):
    df = pd.DataFrame(good[ticker])
    future = df['Close'].shift(-1) #shifts the values back by one
    df.insert(len(df.columns), 'Future', future)
    df.dropna(inplace=True)
    
    with_future[ticker] = df
    
    
    

100%|██████████| 21/21 [00:00<00:00, 260.51it/s]


In [15]:
with_future['TSLA']

,Open,High,Low,Close,Volume,DayOfWeek,Future
Date,,,,,,,
2011-01-27,0.001214,0.008039,0.017842,0.006869,-0.170571,0.75,-0.036517
2011-01-28,0.005659,-0.007974,-0.031798,-0.036517,0.170481,1.00,0.003748
2011-01-31,-0.033360,-0.030547,-0.010526,0.003748,-0.208031,0.00,-0.007884
2011-02-01,0.010811,0.025290,0.001702,-0.007884,-0.147537,0.25,0.001255
2011-02-02,-0.006170,-0.022240,0.005523,0.001255,-0.195394,0.50,-0.012949
...,...,...,...,...,...,...,...
2021-01-19,-0.016667,-0.011513,0.016970,0.022260,-0.345834,0.25,0.006986
2021-01-20,0.024994,0.011176,0.005138,0.006986,0.011783,0.50,-0.006420
2021-01-21,-0.004355,-0.004398,0.004945,-0.006420,-0.197453,0.75,0.001953


In [16]:
import random
from collections import deque

#data is 2D array where each subarray is row



# converts data frame into individual sequences 
# df the dataframe where last column is 'future value'
def to_seq(df):
    sequences = []
    
    # deque for rolling over
    day_sequence = deque(maxlen = 10) # deque for storing the days
    
    for row in df.values:
        # appends values (in list) to day_sequence, excluding future price
        day_sequence.append(row[:-1])
        
        if len(day_sequence) == 10:
            # deque fully populated with 10 days of data
            sequences.append([np.array(day_sequence), row[-1]])
            
    return sequences



train_seq = {} # train sequences
val_seq = {} #  out of sample val (last 5%)

#convert into sequences
for ticker in tqdm(with_future):
    seq = to_seq(with_future[ticker])
    
    split = int(len(seq)*0.05)
    
    train = seq[:-split]  # 95%
    test = seq[-split:]  # 5%
    
    random.shuffle(train)
    random.shuffle(test)
    
    train_seq[ticker] = train
    val_seq[ticker] = test



100%|██████████| 21/21 [00:00<00:00, 26.44it/s]


In [17]:
df.values[0][-1]

-0.027272701657507614

In [1]:
train_seq['TSLA'][1]

NameError: name 'train_seq' is not defined

In [22]:
with open('./data/train-sequences.pkl', 'wb') as f:
    pickle.dump(train_seq, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./data/test-sequences.pkl', 'wb') as f:
    pickle.dump(val_seq, f, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
print(len(train_seq['APHA.TO']))
print(len(val_seq['APHA.TO']))

1504
79


## New Normalization Approach

Using built in pandas methods to use z scoring to make it completely error proof (at least not caused by me)

Will build sequences of size 11, then z-score, set the closing of last row as the future price, and drop the last row.

In [14]:
# df is dataframe of 11 rows
def z_score(df):
    stand = df.copy()
    
    # DAY OF WEEK ALSO GETS STANDARDIZED
    stand['DayOfWeek']=[i.dayofweek for i in df.index]
    
    for column in stand.columns:
        stand[column] = (stand[column] - stand[column].mean()) / stand[column].std()
    
    future = stand['Close'][10]
    
    # return first 10 rows of df, excluding the future row
    return [stand.iloc[0:10].values, future]
    

all_seq = {}

for ticker in tqdm(tickers):
    all_seq[ticker] = []
    
    for i in range(len(col_d[ticker].index)-11):
        sequence = z_score(col_d[ticker].iloc[i:i+11])
        all_seq[ticker].append(sequence)
        
        



100%|██████████| 21/21 [04:40<00:00, 13.34s/it]


In [15]:
all_seq['TSLA'][0]

[array([[ 1.39088772,  1.26831302,  2.26475165,  1.93734709, -0.29332967,
          0.62869461],
        [ 1.64864442,  0.95793595,  0.76886414,  0.34254076, -0.13574952,
          1.32025869],
        [ 0.12050979, -0.22149768,  0.28941161,  0.50026982, -0.36081979,
         -1.44599761],
        [ 0.59920331,  0.72515499,  0.3661273 ,  0.16728764, -0.48723478,
         -0.75443354],
        [ 0.32303229, -0.12838382,  0.61544189,  0.21986399, -0.62995472,
         -0.06286946],
        [-0.30294951, -0.56291024, -0.38181645, -0.32342361, -0.68929236,
          0.62869461],
        [-1.00257546, -0.91984425, -0.24757084, -0.62135349, -0.65626967,
          1.32025869],
        [-1.33398069, -1.55611985, -0.89962226, -1.30484192, -0.29394885,
         -1.44599761],
        [-0.37659366,  1.53213687, -0.66948888,  1.18375407,  2.39925791,
         -0.75443354],
        [ 0.26780247, -0.12838382, -1.07222572, -1.05948421,  1.5021759 ,
         -0.06286946]]),
 -1.0419601487583074]

In [16]:
import random

train = []
test = []

# test:train ratio
RATIO = 0.05 

for ticker in tqdm(tickers):
    seqs = all_seq[ticker]
    
    index = math.floor(len(seqs)*RATIO)
    
    train.extend(seqs[:-index])
    test.extend(seqs[-index:])
    

random.shuffle(train)
random.shuffle(test)

    


100%|██████████| 21/21 [00:00<00:00, 6107.36it/s]


In [17]:
import pickle

with open('./data/new-train.pkl', 'wb') as f:
    pickle.dump(train, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./data/new-test.pkl', 'wb') as f:
    pickle.dump(test, f, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
import pickle

test = []
with open('./data/new-test.pkl', 'rb') as f:
    test = pickle.load(f)
    
train = []
with open('./data/new-train.pkl', 'rb') as f:
    train = pickle.load(f)
    


In [13]:
for t in test:
    seq = t[0]
    assert len(seq) == 10
    
    for r in seq:
        assert len(r) == 6
    

AssertionError: 